In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# 버킷명 읽어오기
import os

BUCKET_NAME = os.environ.get('S3_BUCKET_NAME')

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:3.3.4" pyspark-shell'

In [ ]:
spark = SparkSession.builder \
    .appName("S3JSONAnalysis") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

In [ ]:
AWS_ACCESS_KEY_ID="..."
AWS_SECRET_ACCESS_KEY="..."
AWS_DEFAULT_REGION="..."
REGION_NAME="..."

In [ ]:
import boto3

s3 = boto3.client('s3', 
                  aws_access_key_id=AWS_ACCESS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
                  region_name=REGION_NAME)
obj_list = s3.list_objects(Bucket=BUCKET_NAME, Delimiter='/')
content_list = obj_list['CommonPrefixes']

In [ ]:
from pyspark.sql.functions import regexp_replace

for i, content in enumerate(content_list):
    webtoon_no = content['Prefix'].split('/')[0]
    
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .json(f"s3a://{bucket_name}/{webtoon_no}/*.json")
    
    df\
        .coalesce(1)\
        .write\
        .mode("overwrite")\
        .option("header",True) \
        .csv(f"csv_files/all_comments_{webtoon_no}.csv")
    
    complete_hangul_df = df \
                    .withColumn("sentence", regexp_replace(df["comment"], "[^가-힣\s]+", ""))\
                    .drop("comment")
    
    complete_hangul_df\
        .coalesce(1)\
        .write\
        .mode("overwrite")\
        .json(f"complete_hangul/complete_hangul_output_{webtoon_no}.json")
    
    print(i)